In [1]:
import tensorflow as tf
import numpy as np

# Constant & Variable

**`tf.constant`**(value, dtype=None, shape=None, name='Const', verify_shape=False)
+ tf.constant_initializer

In [2]:
cost = tf.constant([1,2,3,4])
with tf.Session() as sess:
    print(sess.run(cost))

[1 2 3 4]


In [3]:
bias = tf.constant(0.0, dtype=tf.float32, shape=[1,8])
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(bias))

[[0. 0. 0. 0. 0. 0. 0. 0.]]


----------------------------------

**`tf.Variable`**(initial_value, trainable, collections, validate_shape, name, dtype)
+ initial_value : A `Tensor`, or Python object convertible to a `Tensor`, which is the initial value for the Variable.
+ trainable: 
    + If `True`, the default, also adds the variable to the graph collection `GraphKeys.TRAINABLE_VARIABLES`. 
    + This collection is used as the default list of variables to use by the `Optimizer` classes.
+ collections: 
    + List of graph collections keys. 
    + The new variable is added to these collections. Defaults to `[GraphKeys.GLOBAL_VARIABLES]`.
+ validate_shape: 
    + If `False`, allows the variable to be initialized with a value of unknown shape.
    + If `True`, the default, the shape of `initial_value` must be known.

In [4]:
zeros = tf.zeros([3,4])
var = tf.Variable(zeros)
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(var))

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [5]:
weights = tf.Variable(tf.random_normal([2,3], stddev=2))

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(weights))

[[-0.3574394  -3.3459775   0.75897485]
 [ 2.2388954  -1.5581603  -0.53937155]]


# Variable & Placeholder & Fetch

**`Fetch`**
+ 在一个绘画中可以`同时执行多个op`

In [6]:
input1 = tf.constant(3.0)
input2 = tf.constant(2.0)
input3 = tf.constant(5.0)

add = tf.add(input2, input3)
mul = tf.multiply(input1, add)

with tf.Session() as sess:
    result = sess.run([mul, add])  # <<<　Fetch
    print("[mul, add] :", result)

[mul, add] : [21.0, 7.0]


----------------------------------

**`Feed`**
+ **在运行时, 才传入相应的值**
+ 占位符 : tf.placeholder(tf.float32)
+ feed_dict = {.. value ..}

In [7]:
input1 = tf.placeholder(tf.float32)  # 创建 float32 类型的占位符
input2 = tf.placeholder(tf.float32)

output = tf.multiply(input1, input2)

with tf.Session() as sess:
    res = sess.run(output, feed_dict={
        input1:[7.0],
        input2:[2.]
    })
    print(res)

[14.]


In [8]:
w1 = tf.Variable(tf.random_normal([3,2], stddev=1))
w2 = tf.Variable(tf.random_normal([1,3], stddev=1))

x = tf.placeholder(tf.float32, shape=(2,3), name="input")
a = tf.matmul(w1, x)
y = tf.matmul(w2, a)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    res = sess.run(y, feed_dict={
        x : [[0.7,0.5,0.5],[0.9,0.4,0.8]]
    })
    print(res)

[[1.898734  1.0420545 1.5595341]]


# get_variable & variable_scope("", reuse= )

**`tf.get_variable`**(name, shape=None, dtype=None, initializer=None, regularizer=None, trainable=None, collections=None)
+ 初始化更方便
```
W = tf.get_variable("W", shape=[784, 256], initializer=tf.contrib.layers.xavier_initializer())
```

**`tf.variable_scope`**(name_or_scope, reuse=None)

+ 当神经网络的结构更加复杂, 参数更多时, 就需要一个更好的方式来传递和管理神经网络中的参数
+ Tensorflow 提供了`通过变量名称`来创建或者获取一个变量的机制
+ 通过这个机制, 在不同的函数中可以直接通过变量的名字来使用变量, 而不需要将变量通过参数的形式到处传递

+ tf.get_variable() 用于创建变量时, 它和 tf.Variable() 的功能是基本等价的
    + 对于 tf.Variable 函数, 变量名称是一个可选的参数
    + 对于 tf.get_variable 函数, 变量名称是一个必选参数
```
    v = tf.get_variable("var_00", shape=[1], initializer=tf.constant_initializer(1.0))
    v = tf.Variable(tf.constant(1.0, shape=[1]), name='var_00')
```

----

**`tf.get_variable 工作机制`**

+ 当 `tf.get_variable_scope().reuse == False`，调用该函数会创建新的变量
```
      with tf.variable_scope("foo"):
          v = tf.get_variable("v", [1])
      assert v.name == "foo/v:0"
```
    　　

+ 当 `tf.get_variable_scope().reuse == True`，调用该函数会重用已经创建的变量
```
      with tf.variable_scope("foo"):
          v = tf.get_variable("v", [1])
      with tf.variable_scope("foo", reuse=True):
          v1 = tf.get_variable("v", [1])
      assert v1 is v
```

----

**`tf.variable_scope 工作机制`**

+ `tf.variable_scope()`用来指定变量的作用域，作为变量名的前缀，支持嵌套
```
    with tf.variable_scope("foo"):
        with tf.variable_scope("bar"):
            v = tf.get_variable("v", [1])
    assert v.name == "foo/bar/v:0"
```

+ 当前环境的作用域可以通过函数 `tf.get_variable_scope()` 获取
+ 并且 **`reuse flag`** 可以通过调用 `reuse_variables()` 设置为True
```
    with tf.variable_scope("foo"):
        v = tf.get_variable("v", [1])
        tf.get_variable_scope().reuse_variables()
        v1 = tf.get_variable("v", [1])
    assert v1 is v
```

----

**`Sharing variables`**
+ 使用`tf.variable_scope()`来生成一个上下文管理器, 在这个上下文管理器中, 通过`tf.get_variable()`来`创建/获取`变量
+ **`tf.variable_scope(., reuse=True)`** & **`tf.get_variable()`** 获取
    + 获取 : 如果变量不存在, 则tf.get_variable()将报错
    + 创建 : 如果同名的变量已经存在, 则tf.get_variable()函数将报错

**`reuse = tf.AUTO_REUSE`**
+ 自动复用
+ 如果变量存在则复用，不存在则创建
+ 避免了代码的繁琐

In [9]:
def foo():
    with tf.variable_scope("foo", reuse=tf.AUTO_REUSE):
        v = tf.get_variable("v", [1], initializer=tf.ones_initializer)
    return v

v1 = foo()  # Creates v.
v2 = foo()  # Gets the same, existing v.

print(v1.name)
print(v2.name)
assert v1 == v2

foo/v:0
foo/v:0


**`Example - 00`**

In [10]:
with tf.Session() as sess:
    with tf.variable_scope("Tyrion"):
        var = tf.get_variable('var_01', [1], initializer=tf.zeros_initializer)
        print(var.name)

    with tf.variable_scope("Tyrion", reuse=True):
        get_var = tf.get_variable("var_01")
        tf.global_variables_initializer().run()
        print(sess.run(get_var))

Tyrion/var_01:0
[0.]


**`Example - 01`**

In [11]:
with tf.variable_scope("foo"):
    with tf.variable_scope("bar"):
        v = tf.get_variable("var_02", [1])
        assert v.name == "foo/bar/var_02:0"

**`Example - 02`**

In [12]:
def conv_relu(input, kernel_shape, bias_shape):
    weights = tf.get_variable("weights", kernel_shape, initializer=tf.random_normal_initializer())
    biases = tf.get_variable("biases", bias_shape, initializer=tf.constant_initializer(0.0))
    conv = tf.nn.conv2d(input, weights, strides=[1, 1, 1, 1], padding='SAME')
    return tf.nn.relu(conv + biases)

In [13]:
# 需要两个卷积层，可以通过 tf.variable_scope() 指定作用域进行区分
# 如 with tf.variable_scope("conv1") 指定了第一个卷积层作用域为conv1
# 在这个作用域下有两个变量 weights 和 biases

def my_image_filter(input_images):
    with tf.variable_scope("conv1"):
        # Variables created here will be named "conv1/weights", "conv1/biases".
        relu1 = conv_relu(input_images, [5, 5, 32, 32], [32])
    with tf.variable_scope("conv2"):
        # Variables created here will be named "conv2/weights", "conv2/biases".
        return conv_relu(relu1, [5, 5, 32, 32], [32])

+ 在 image_filters 这个作用域, 重复使用第一张图片输入时创建的变量，调用函数reuse_variables()
```
with tf.variable_scope("image_filters") as scope:
    result1 = my_image_filter(image1)
    scope.reuse_variables()
    result2 = my_image_filter(image2)
```

**`Example - 03`**

In [14]:
def func(in_put, in_channel, out_channel):
    with tf.variable_scope(name_or_scope='', reuse=tf.AUTO_REUSE): ### 改动部分 ###
        weights = tf.get_variable(name="weights", shape=[2, 2, in_channel, out_channel], 
                                  initializer=tf.contrib.layers.xavier_initializer_conv2d())
        convolution = tf.nn.conv2d(input=in_put, filter=weights, strides=[1, 1, 1, 1], padding="SAME")
        return convolution

def main():
    with tf.Graph().as_default():
        input_x = tf.placeholder(dtype=tf.float32, shape=[1, 4, 4, 1])
        for _ in range(5):    # 重复调用
            output = func(input_x, 1, 1)
            with tf.Session() as sess:
                sess.run(tf.global_variables_initializer())
                _output = sess.run([output], feed_dict={
                    input_x : np.random.uniform(low=0, high=255, size=[1, 4, 4, 1])
                })
                # print(_output)

if __name__ == "__main__":
    main()

--------------------------------------------------------

# generator

+ **`Use already have value`**
+ **`Fix Value`**
+ **`Sequence`**
+ **`Random Value`**
+ **`initializer`**

**`Tensor.initialized_value()`**

In [15]:
weights = tf.Variable(tf.random_normal([2,3], stddev=2))

weights_ = tf.Variable(weights.initialized_value() * 2.0)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(weights))
    print()
    print(sess.run(weights_))

[[-1.1493891   0.60526866 -2.0620763 ]
 [-3.920175   -0.45096406 -3.321645  ]]

[[-2.2987783  1.2105373 -4.1241527]
 [-7.84035   -0.9019281 -6.64329  ]]


**`Fix Value`**
+ tf.`fill`(dims, value, name=None)
+ tf.`zeros`(shape, dtype=tf.float32, name=None) / tf.`ones`( )
+ tf.`zeros_like`(tensor, dtype=None, name=None, optimize=True) / tf.`ones_like`( )

In [16]:
f_one = tf.fill([3,4], 1)

with tf.Session() as sess:
    print(sess.run(f_one))

[[1 1 1 1]
 [1 1 1 1]
 [1 1 1 1]]


In [17]:
ones = tf.ones([4,4])
zeros = tf.zeros([3,4])
zeros_l = tf.zeros_like(ones)

with tf.Session() as sess:
    tf.global_variables_initializer().run()
    print(sess.run(zeros))
    print(sess.run(zeros_l))
    print()
    print(zeros.get_shape())
    print(zeros_l.get_shape())

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]

(3, 4)
(4, 4)


**`Sequence`**

+ **`tf.linspace(start, stop, num, name=None)`**
    + 在 [start,stop] 范围内产生 num 个数的等差数列
    + start & stop 要用浮点数表示
    
+ **`tf.range(start, limit=None, delta=1, dtype=None, name='range')`**
    + 在 [start,limit) 范围内以步进值 delta 产生等差数列

In [18]:
x = tf.linspace(start=1.0, stop=5.0, num=5, name=None)
y = tf.range(start=1,limit=5, delta=1)
with tf.Session() as sess:
    print(sess.run(x))
    print(sess.run(y))

[1. 2. 3. 4. 5.]
[1 2 3 4]


**`Random Value`**

+ **`tf.random_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`**
    + Outputs random values from a normal distribution.
    + 正态分布

+ **`tf.truncated_normal(shape, mean=0.0, stddev=1.0, dtype=tf.float32, seed=None, name=None)`**
    + Outputs random values from a truncated normal distribution.
    + 截断正态分布，只保留 [mean-2*stddev, mean+2*stddev] 内的随机数

+ **`tf.random_uniform(shape, minval=0, maxval=None, dtype=tf.float32, seed=None, name=None)`**
    + Outputs random values from a uniform distribution.
    + 均匀分布 : [minval, maxval)

**`initializer`**

+ tf.ones_initializer(dtype=tf.float32)
+ tf.zeros_initializer(dtype=tf.float32)
+ tf.constant_initializer(value=0, dtype=tf.float32, verify_shape=False)
+ tf.random_uniform_initializer(minval=0, maxval=None, seed=None, dtype=tf.float32)
+ tf.random_normal_initializer(mean=0.0, stddev=1.0, seed=None, dtype=tf.float32)
+ tf.truncated_normal_initializer(mean=0.0, stddev=1.0, seed=None, dtype=tf.float32)

# Graph & Session

In [19]:
sess = tf.Session()
# ...
sess.close()

In [20]:
with tf.Session() as sess:
    pass

In [21]:
+ **Operation 需要在 Graph 中执行**
+ **Graph 需要在 Session 中执行**

SyntaxError: invalid syntax (<ipython-input-21-add701d4640e>, line 1)

In [ ]:
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('v', initializer=tf.zeros(shape=[1]))

g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', initializer=tf.ones(shape=[1]))

In [ ]:
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable('v')), "in Graph-01")

with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope("", reuse=True):
        print(sess.run(tf.get_variable('v')), "in Graph-02")

In [ ]:
g = tf.Graph()
with g.as_default():
    zero = tf.Variable(initial_value=tf.zeros(shape=[1]), name='Var')
    one = tf.Variable(initial_value=tf.ones(shape=[1,2,3]), name='Var1')
    
with tf.Session(graph=g) as sess:
    init = tf.global_variables_initializer()
    sess.run(init)
    print(sess.run(zero))
    print(sess.run(one))

In [ ]:
# graph

In [ ]:
**`tf.get_default_graph() : 获取当前默认计算图`**

In [ ]:
a = tf.constant(1)

In [ ]:
a.graph is tf.get_default_graph()

In [ ]:
# Session

In [ ]:
**`tf.Tensor.eval() / sess.run(t)`**

In [ ]:
+ Basic Usage

In [ ]:
test = tf.constant([1,2,3], name='test')

In [ ]:
sess = tf.Session()
with sess.as_default():
    print(sess is tf.get_default_session())
    
    print(test.eval() == sess.run(test))
    print(test.eval())
    print(sess.run(test))

In [ ]:
+ **`sess.run() 可以一步获取多个tensor中的值`**

In [ ]:
a = tf.constant([1, 2, 3], name='a')
b = tf.constant([3, 4, 5], name='b')

In [ ]:
add = tf.add(a, b, name='add')
mul = tf.multiply(a, b, name='mul')

In [ ]:
with tf.Session() as sess:
    print(sess.run([add, mul]))
    print(add.eval(session=sess), mul.eval(session=sess))

# collection

In [ ]:
**`tf.GraphKeys`**

In [ ]:
+ tf.GraphKeys.**GLOBAL_VARIABLES**
    + 默认的 Variable 对象集合，在分布式环境共性
    + 使用场景：持久化 Tensorflow 模型


+ tf.GraphKeys.**LOCAL_VARIABLES**
    + 每台计算机的局部变量
    + 通常用于临时变量，如：计数器
    + 使用 tf.contrib.framework.local_variable 添加到此集合


+ tf.GraphKeys.**MODEL_VARIABLES**
    + 在构建模型时，所有用于正向传播的 Variable 都将添加到这里
    + 使用 tf.contrib.framework.model_variable 添加到此集合


+ tf.GraphKeys.**TRAINABLE_VARIABLES**
    + 将有优化器训练的变量
    + tf.trainable_variables()


+ tf.GraphKeys.**SUMMARIES**
    + 日志生成相关的张量
    + Tensorflow 计算可视化


+ tf.GraphKeys.**MOVING_AVERAGE_VARIABLES**
    + 所有计算了滑动平均值的变量
    + tf.moving_average_variables


+ tf.GraphKeys.**REGULARIZATION_LOSSES**
    + 收集在图构造期间正则化损失

In [ ]:
**`tf.GraphKeys 使用`**

# operation / op

In [ ]:
# Property

In [ ]:
tf.shape(Tensor)
# tf.shape 函数本身也是返回一个张量，因而，需要用 sess.run() 来得到具体的值

In [ ]:
Tensor.get_shape()

In [ ]:
tf.reshape(tensor, shape, name=None)
&
-1

In [ ]:
t = tf.constant([1,2,3,4,5,6,7,8,9,10,11,12])
t1 = tf.reshape(t, [-1,3])
with tf.Session() as sess:
    print(sess.run(t1))

In [ ]:
a = tf.constant([[1,2,3],[1,2,3],[1,2,3],[1,2,3]], name='a')
b = tf.constant([[3,4,5],[3,4,5],[3,4,5],[3,4,5]], name='b')

result = tf.add(a,b, name='add')
print(result)

In [ ]:
result.get_shape()

In [ ]:
assign

In [ ]:
**`Implement Loop add one`**

In [ ]:
count = tf.Variable(0, name='counter')
one = tf.constant(1)
new_value = tf.add(count, one)

In [ ]:
**赋值op : tf.assign(old, new)**

In [ ]:
update = tf.assign(count, new_value)

In [ ]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    print(sess.run(count))
    for _ in range(5):
        sess.run(update)
        print(sess.run(count))

In [ ]:
product = tf.matmul(m1, m2)

In [ ]:
print(product)

In [ ]:
+ **`tf.random_shuffle()`**
tf.one_hot